##### Pipeline for using pre-trained U-Net for boundary identification 

In [1]:
#imports
from module_prediction import make_patches, setup_directories, deploy_model_on_folder, constructing_map_mask
from datetime import datetime
import shutil
import os

Model architecture functions loaded successfully


In [2]:
# configuration 
map_name = "example"

input_map = f"images-for-prediction/{map_name}/{map_name}.tif"
output_dir = f"images-for-prediction/{map_name}/overlapping_patches"

patch_size = 256
image_channels = 1 #map converted to greyscale
batch_size = 10
threshold = 0.5 #prediction confidence
image_format = ['.png', '.jpg', '.jpeg', '.tiff', '.tif', '.bmp']
# File paths
input_folder = f'images-for-prediction/{map_name}/overlapping_patches'
output_folder = f'images-for-prediction/{map_name}/predicted_masks_overlapping'
model_weights_path = 'model-weights-and-training-history/model_weights.weights.h5'

In [10]:
# turn target map into patches
make_patches(map_name, input_map, output_dir, patch_size)

"process finished. Patches with 50% overlap saved to 'images-for-prediction/example/overlapping_patches'"

In [11]:
# Setup directories and get image list
output_dirs, image_files = setup_directories(input_folder, output_folder, image_format, batch_size)


Found 28 images in input folder: images-for-prediction/example/overlapping_patches
First few files: ['example_0_0.png', 'example_0_1.png', 'example_0_2.png', 'example_0_3.png', 'example_0_4.png']
... and 23 more files

Setup completed!
Total images to process: 28
Estimated batches: 3


In [12]:
# predict on the patches

print("=" * 50)
print("STARTING MODEL DEPLOYMENT")
print("=" * 50)

start_time = datetime.now()
results, trained_model = deploy_model_on_folder(
    image_files,
    output_dirs,
    model_weights_path,
    patch_size,
    image_channels,
    batch_size,
    threshold
)
end_time = datetime.now()

processing_time = end_time - start_time
print(f"\nTotal processing time: {processing_time}")

if results:
    successful = sum(1 for r in results if r['processed'])
    failed = len(results) - successful
    print(f"Successfully processed: {successful}/{len(results)} images")
    if failed > 0:
        print(f"Failed to process: {failed} images")


STARTING MODEL DEPLOYMENT
Loading trained model...
Model loaded successfully!

Starting batch processing...
Total images: 28
Batch size: 10
Total batches: 3


Processing batch 3/3: 100%|██████████| 28/28 [00:02<00:00,  9.55it/s]


Batch processing completed!

Total processing time: 0:00:03.338516
Successfully processed: 28/28 images


In [3]:
constructing_map_mask(map_name, input_map, output_folder, patch_size)

Original map dimensions: 811 x 444
Original CRS: EPSG:27700
Original bounds: BoundingBox(left=286954.98952713405, bottom=147007.4379671646, right=287360.48952713405, top=147229.4379671646)
Found 28 mask patches
Patch grid indices: rows=4, cols=7
Detected patch_size=256, using step=128
Padded dimensions: 1024 x 640
Original padding applied: height=196, width=213
Will crop final image to: 811 x 444
Mask patches are grayscale
Patches stitched into stitched_array.
Georeferenced mask saved as: images-for-prediction/example/example_stitched_mask.tif
Output dimensions: 811 x 444
Output CRS: EPSG:27700
Output channels: 1

Verification:
Saved file CRS: EPSG:27700
Saved file bounds: BoundingBox(left=286954.98952713405, bottom=147007.4379671646, right=287360.48952713405, top=147229.4379671646)
Saved file shape: (444, 811)


In [ ]:
# delete patch folder once finished

for folder in [output_dir, output_folder]:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Deleted folder: {folder}")
    else:
        print(f"Folder not found: {folder}")

Deleted folder: images-for-prediction/example/overlapping_patches
Deleted folder: images-for-prediction/example/predicted_masks_overlapping



***
